In [ ]:
%pip install polars
%pip install tiktoken

In [ ]:
import os
import getpass
import pandas as pd 
import polars as pl 
import numpy as np
import os
import re
import sys
import time

from polars import col, lit, when

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

In [ ]:
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI

In [ ]:
# Load pandas dataframe from csv 
# Convert to polars object 
# View the first 5 rows 

df = pd.read_csv('data/Metro_zori_sm_month.csv')
df = pl.from_pandas(df)
df.head(5)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA
llm = OpenAI(temperature=0)

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

In [ ]:
from langchain.document_loaders import TextLoader
loader = CSVLoader(file_path='data/Metro_zori_sm_month.csv')

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)


# Keep 100 elements from texts 
texts = texts[:5]

embeddings = OpenAIEmbeddings()
state_of_union_store = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [ ]:
texts

In [ ]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)
vectorstore_info = VectorStoreInfo(
    name="state_of_union_address",
    description="a dataframe",
    vectorstore=state_of_union_store
)
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)
agent_executor = create_vectorstore_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [ ]:
agent_executor.run("How many rows in dataframe?")